In [17]:
#carregamento de bibliotecas

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time
from six import StringIO
from IPython.display import Image  
import pydotplus
import import_ipynb
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import itertools
from controllers.utils.utils import show_title, mostrar_dataframe_interativo

show_title("bibliotecas carregadas")


In [18]:
#configuracoes e variaveis iniciais

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.6f}'.format
iterations_count = 10;
algoritms = ['knn', 'dt', 'svm', 'mlp']

show_title("configurações iniciais concluídas")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#carregamendo dos dados

df = pd.read_csv("data.csv")

show_title(f"dados carregados: {df['cid'].count()} registros")


In [20]:
#ETL

df = df.drop(columns=['pidnum']) #pidnum é a coluna de identificação do paciente, não tem significado para o treinamento

# Identifica e remove colunas com valores iguais em todas as linhas
for col in df.columns:
    if df[col].nunique() == 1:        
        df = df.drop(columns=[col])
        print(f"coluna {col} excluida")
        
show_title("ETL concluido")

coluna zprior excluida


In [21]:
# Embaralhando as linhas do DataFrame

df = df.sample(frac=1, random_state=None).reset_index(drop=True)
show_title("Dados embaralhados")
mostrar_dataframe_interativo(df)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [22]:
# Extratificação 

X = df.drop(columns=['cid'])  # Variáveis independentes (todas as colunas menos a 'cid')
y = df['cid']  # Variável dependente (alvo)

show_title("Variaveis independentes")
mostrar_dataframe_interativo(X)



IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [23]:
show_title("Variavel dependente (target)")
mostrar_dataframe_interativo(y)

IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [24]:
# Contagem das classes antes da divisão

show_title("Distribuição original das classes em 'cid':")
print(y.value_counts(normalize=True))  # Proporção das classes antes da divisão


cid
0   0.756428
1   0.243572
Name: proportion, dtype: float64


In [25]:
# Primeiro, dividir em treino (50%) e restante (50% para validação + teste)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

show_title("dados splitados para treinamento")

In [26]:
# Contagem das classes no conjunto de treino

show_title("Distribuição das classes no conjunto de treino (y_train):")
print(y_train.value_counts(normalize=True))  # Proporção das classes no treino


cid
0   0.756782
1   0.243218
Name: proportion, dtype: float64


In [27]:
# Segunda divisão: validação (25%) e teste (25%)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

show_title("dados de validacao e teste splitados")


In [28]:
# Contagem das classes nos conjuntos de validação e teste

show_title("Distribuição das classes no conjunto de validação (y_val):")
print(y_val.value_counts(normalize=True))  # Proporção das classes na validação
print()

show_title("Distribuição das classes no conjunto de teste (y_test):")
print(y_test.value_counts(normalize=True))  # Proporção das classes no teste
print()


cid
0   0.755140
1   0.244860
Name: proportion, dtype: float64



cid
0   0.757009
1   0.242991
Name: proportion, dtype: float64



In [29]:
#funcao para rodar o classificar e armazenar os resultados

def update_results(results,algoritm,iteration,y_val, classifier):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_val)  # Usando X_val e y_val para validação
    iteration['accuracy'] = metrics.accuracy_score(y_val, y_pred)
    iteration['confusion_matrix'] = metrics.confusion_matrix(y_val, y_pred).tolist()
    iteration['elapsed_time'] = time.time() - iteration['start_time']
    
    results[algoritm]['iterations'].append(iteration)

    # Atualizar melhores e piores índices e valores de acurácia e tempo
    if not results[algoritm]['best_accuracy'] or iteration['accuracy'] > results[algoritm]['best_accuracy']['accuracy']:
        results[algoritm]['best_accuracy'] = iteration

    if not results[algoritm]['worst_accuracy'] or iteration['accuracy'] < results[algoritm]['worst_accuracy']['accuracy']:
        results[algoritm]['worst_accuracy'] = iteration
        
    if not results['best_metrics']['best_accuracy'] or results['best_metrics']['best_accuracy']['accuracy'] < iteration['accuracy']: 
        results['best_metrics']['best_accuracy'] = dict(iteration)
        results['best_metrics']['best_accuracy']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_accuracy'] or results['worst_metrics']['worst_accuracy']['accuracy'] > iteration['accuracy']: 
        results['worst_metrics']['worst_accuracy'] = dict(iteration)
        results['worst_metrics']['worst_accuracy']['algoritm'] = algoritm

    if not results[algoritm]['best_time'] or iteration['elapsed_time'] < results[algoritm]['best_time']['elapsed_time']:
        results[algoritm]['best_time'] = iteration

    if not results[algoritm]['worst_time'] or iteration['elapsed_time'] > results[algoritm]['worst_time']['elapsed_time']:
        results[algoritm]['worst_time'] = iteration
        
    if not results['best_metrics']['best_time'] or results['best_metrics']['best_time']['elapsed_time'] > iteration['elapsed_time']: 
        results['best_metrics']['best_time'] = dict(iteration)
        results['best_metrics']['best_time']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_time'] or results['worst_metrics']['worst_time']['elapsed_time'] < iteration['elapsed_time']: 
        results['worst_metrics']['worst_time'] = dict(iteration)
        results['worst_metrics']['worst_time']['algoritm'] = algoritm

show_title("função de classificacao e atualização de resultados criada")

In [30]:
# Dicionários para armazenar resultados de validação e teste

validation_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

test_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

for alg in algoritms:
    validation_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}
    test_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}

show_title("dicionario de resultados de validação e teste criado")

In [31]:
#LOOP DE EXECUSSÕES DE VALIDAÇÃO (LOOP 1)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':{
            'k':np.random.randint(1, 20), # Valor de K aleatório 
            'weights': np.random.choice(['uniform', 'distance']) # Pesos aleatórios
        },
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(validation_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':{
            'max_depth': np.random.randint(1, 100),  # Aleatório entre 
            'min_samples_split': np.random.randint(2, 30),  # Aleatório entre 2 e 10 (influencia na poda)
            'criterion': np.random.choice(['gini', 'entropy']),  # Critério aleatório
            'ccp_alpha': np.random.uniform(0.0, 0.1)   # Parâmetro de complexidade de poda entre 0 e 0.05 (influencia na poda)
        },
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(validation_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':{
            'C': np.random.uniform(0.1, 1.0),  # Valor de erro aleatório (C)
            'kernel': np.random.choice(['rbf', 'poly'])  # Tipo de Kernel aleatório (linear excluido por ser mais demorado)
        },
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(validation_results,'svm',svm_iteration,y_val,svm_clf) 

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':{
            'epochs': np.random.randint(100, 301),  # Número de épocas aleatório entre 100 e 300
            'learning_rate': np.random.uniform(0.001, 0.1),  # Taxa de aprendizagem aleatória
            'hidden_layers': np.random.randint(1, 6),  # Número de camadas escondidas aleatório entre 1 e 5
            'activation': np.random.choice(['identity', 'logistic', 'tanh', 'relu']),  # Função de ativação aleatória
            'tol': 1e-2  # Tolerância um pouco maior para convergência
        },
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'], tol=mlp_iteration['parameters']['tol'])
    update_results(validation_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    validation_results['iterations'] += 1

validation_results['elapsed_time'] = time.time() - validation_results['start_time']

show_title("loop de validacao concluido")


In [32]:
# Exibir resultados de validação
show_title("mostrar json ? (descomente)")
print(json.dumps(validation_results, indent=4))

NameError: name 'json' is not defined

In [33]:
#tabulação dos resultados da validação e calculo das medias

validation_accuracy_table = {
    'repetição':[]
}
validation_time_table = {
    'repetição':[]
}
validation_accuracy_avgs = {}
validation_time_avgs = {}

for alg in algoritms:
    validation_accuracy_table[alg] = []
    validation_accuracy_avgs[alg] = 0
    validation_time_table[alg] = []
    validation_time_avgs[alg] = 0

for i in range(iterations_count):
    validation_accuracy_table['repetição'].append(i+1)  
    validation_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(validation_results[alg]['iterations']) > 0:
            validation_accuracy_table[alg].append(validation_results[alg]['iterations'][i]['accuracy'])
            validation_accuracy_avgs[alg] += validation_results[alg]['iterations'][i]['accuracy']
            validation_time_table[alg].append(validation_results[alg]['iterations'][i]['elapsed_time'])
            validation_time_avgs[alg] += validation_results[alg]['iterations'][i]['elapsed_time']
        else:
            validation_accuracy_table[alg].append(0)
            validation_time_table[alg].append(float('inf'))

validation_accuracy_table['repetição'].append('AVG')
validation_accuracy_table['repetição'].append('WORST')
validation_accuracy_table['repetição'].append('BEST')
validation_time_table['repetição'].append('AVG')
validation_time_table['repetição'].append('WORST')
validation_time_table['repetição'].append('BEST')
for alg in algoritms:
    validation_accuracy_avgs[alg] = validation_accuracy_avgs[alg] / iterations_count
    validation_time_avgs[alg] = validation_time_avgs[alg] / iterations_count
    validation_accuracy_table[alg].append(validation_accuracy_avgs[alg])
    validation_accuracy_table[alg].append(validation_results[alg]['worst_accuracy']['accuracy'])
    validation_accuracy_table[alg].append(validation_results[alg]['best_accuracy']['accuracy'])
    validation_time_table[alg].append(validation_time_avgs[alg])
    validation_time_table[alg].append(validation_results[alg]['worst_time']['elapsed_time'])
    validation_time_table[alg].append(validation_results[alg]['best_time']['elapsed_time'])

validation_accuracy_table = pd.DataFrame(validation_accuracy_table)
validation_time_table = pd.DataFrame(validation_time_table)

print()
show_title("Medias de acuracias na validação")
print(validation_accuracy_table)
#mostrar_dataframe_interativo(validation_accuracy_table,min(len(validation_accuracy_table),iterations_count)+1,min(len(validation_accuracy_table),iterations_count)+1)
print()
show_title("Medias de tempos na validação")
print(validation_time_table)
#mostrar_dataframe_interativo(validation_time_table,min(len(validation_time_table),iterations_count)+1,min(len(validation_time_table),iterations_count)+1)

   repetição      knn       dt      svm      mlp
0          1 0.814953 0.861682 0.844860 0.755140
1          2 0.813084 0.837383 0.859813 0.364486
2          3 0.835514 0.833645 0.857944 0.755140
3          4 0.841121 0.833645 0.846729 0.755140
4          5 0.809346 0.863551 0.859813 0.857944
5          6 0.809346 0.829907 0.844860 0.755140
6          7 0.835514 0.833645 0.837383 0.755140
7          8 0.842991 0.863551 0.856075 0.755140
8          9 0.828037 0.829907 0.813084 0.803738
9         10 0.835514 0.833645 0.846729 0.867290
10       AVG 0.826542 0.842056 0.846729 0.742430
11     WORST 0.809346 0.829907 0.813084 0.364486
12      BEST 0.842991 0.863551 0.859813 0.867290



   repetição      knn       dt      svm      mlp
0          1 0.089664 0.019858 0.136833 0.056649
1          2 0.149601 0.025930 0.139627 0.108709
2          3 0.145617 0.020938 0.100244 0.041887
3          4 0.088290 0.024112 0.078858 0.057775
4          5 0.138630 0.019946 0.103723 0.149601
5          6 0.152592 0.025931 0.079302 0.060705
6          7 0.153589 0.021941 0.063831 0.110703
7          8 0.086768 0.019947 0.120677 0.053856
8          9 0.132846 0.022939 0.109225 0.103167
9         10 0.138144 0.020104 0.104331 0.142132
10       AVG 0.127574 0.022165 0.103665 0.088518
11     WORST 0.153589 0.025931 0.139627 0.149601
12      BEST 0.086768 0.019858 0.063831 0.041887


In [34]:
#LOOP DE EXECUSSÕES DE TESTE (LOOP 2)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':validation_results['knn']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(test_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':validation_results['dt']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(test_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':validation_results['svm']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(test_results,'svm',svm_iteration,y_val,svm_clf)

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':validation_results['mlp']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'],tol=mlp_iteration['parameters']['tol'])
    update_results(test_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    test_results['iterations'] += 1

test_results['elapsed_time'] = time.time() - test_results['start_time']

show_title("loop de testes concluido")

In [35]:
# Exibir resultados de teste
show_title("mostrar json ? (descomente)")
print(json.dumps(test_results, indent=4))

NameError: name 'json' is not defined

In [36]:
#tabulação dos resultados da testes e calculo das medias

test_accuracy_table = {
    'repetição':[]
}
test_time_table = {
    'repetição':[]
}
test_accuracy_avgs = {}
test_time_avgs = {}

for alg in algoritms:
    test_accuracy_table[alg] = []
    test_accuracy_avgs[alg] = 0
    test_time_table[alg] = []
    test_time_avgs[alg] = 0

for i in range(iterations_count):
    test_accuracy_table['repetição'].append(i+1)  
    test_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(test_results[alg]['iterations']) > 0:
            test_accuracy_table[alg].append(test_results[alg]['iterations'][i]['accuracy'])
            test_accuracy_avgs[alg] += test_results[alg]['iterations'][i]['accuracy']
            test_time_table[alg].append(test_results[alg]['iterations'][i]['elapsed_time'])
            test_time_avgs[alg] += test_results[alg]['iterations'][i]['elapsed_time']
        else:
            test_accuracy_table[alg].append(0)
            test_time_table[alg].append(float('inf'))

test_accuracy_table['repetição'].append('AVG')
test_accuracy_table['repetição'].append('WORST')
test_accuracy_table['repetição'].append('BEST')
test_time_table['repetição'].append('AVG')
test_time_table['repetição'].append('WORST')
test_time_table['repetição'].append('BEST')
for alg in algoritms:
    test_accuracy_avgs[alg] = test_accuracy_avgs[alg] / iterations_count
    test_time_avgs[alg] = test_time_avgs[alg] / iterations_count
    test_accuracy_table[alg].append(test_accuracy_avgs[alg])
    test_accuracy_table[alg].append(test_results[alg]['worst_accuracy']['accuracy'])
    test_accuracy_table[alg].append(test_results[alg]['best_accuracy']['accuracy'])
    test_time_table[alg].append(test_time_avgs[alg])
    test_time_table[alg].append(test_results[alg]['worst_time']['elapsed_time'])
    test_time_table[alg].append(test_results[alg]['best_time']['elapsed_time'])

test_accuracy_table = pd.DataFrame(test_accuracy_table)
test_time_table = pd.DataFrame(test_time_table)

print()
show_title("Medias de acuracias no teste")
print(test_accuracy_table)
#mostrar_dataframe_interativo(test_accuracy_table,min(len(test_accuracy_table),iterations_count)+1,min(len(test_accuracy_table),iterations_count)+1)
print()
show_title("Medias de tempos no teste")
print(test_time_table)
#mostrar_dataframe_interativo(test_time_table,min(len(test_time_table),iterations_count)+1,min(len(test_time_table),iterations_count)+1)

   repetição      knn       dt      svm      mlp
0          1 0.842991 0.863551 0.859813 0.816822
1          2 0.842991 0.863551 0.859813 0.788785
2          3 0.842991 0.863551 0.859813 0.803738
3          4 0.842991 0.863551 0.859813 0.766355
4          5 0.842991 0.863551 0.859813 0.807477
5          6 0.842991 0.863551 0.859813 0.622430
6          7 0.842991 0.863551 0.859813 0.871028
7          8 0.842991 0.863551 0.859813 0.803738
8          9 0.842991 0.863551 0.859813 0.796262
9         10 0.842991 0.863551 0.859813 0.809346
10       AVG 0.842991 0.863551 0.859813 0.788598
11     WORST 0.842991 0.863551 0.859813 0.622430
12      BEST 0.842991 0.863551 0.859813 0.871028



   repetição      knn       dt      svm      mlp
0          1 3.835701 0.024827 0.161568 0.169650
1          2 3.257056 0.024950 0.143539 0.182474
2          3 3.919680 0.024933 0.190064 0.202847
3          4 2.755792 0.024716 0.185971 0.176809
4          5 2.172762 0.024836 0.145610 0.264877
5          6 0.708420 0.025882 0.153456 0.194015
6          7 0.199360 0.025931 0.145853 0.120105
7          8 0.101235 0.024934 0.138630 0.215467
8          9 0.095397 0.022487 0.123986 0.215419
9         10 0.094747 0.024933 0.130024 0.257743
10       AVG 1.714015 0.024843 0.151870 0.199941
11     WORST 3.919680 0.025931 0.190064 0.264877
12      BEST 0.094747 0.022487 0.123986 0.120105


In [37]:
#Aplicando estatistica de Kruskal-Wallis
df_statics = validation_accuracy_table.iloc[:-3] #elimina as 3 ultimas linhas, que contem as medias, melhores e piores

kruskal_result = kruskal(df_statics['knn'], df_statics['dt'], df_statics['svm'], df_statics['mlp'])
show_title("Resultado do teste de Kruskal-Wallis:")
print("Estatística:", kruskal_result.statistic)
print("p-valor:", kruskal_result.pvalue)

alpha = 0.05
if kruskal_result.pvalue < alpha:
    print('Isso indica que os valores das acuracias apuradas são significativamente diferentes.')
else:
    print('Isso indica que os valores das acuracias apuradas não são significativamente diferentes.')

Estatística: 13.776174306734594
p-valor: 0.003226203916811015
Isso indica que os valores das acuracias apuradas são significativamente diferentes.


In [38]:
# # Comparação par-a-par com Mann-Whitney

show_title("\nTeste de Mann-Whitney (comparação dois a dois):")
classifiers = {
    'MLP': df_statics['mlp'],
    'DT': df_statics['dt'],
    'SVM': df_statics['svm'],
    'KNN': df_statics['knn']
}
alpha = 0.05

for (name1, acc1), (name2, acc2) in itertools.combinations(classifiers.items(), 2):
    mannwhitney_result = mannwhitneyu(acc1, acc2, alternative='two-sided')
    print(f"Comparação {name1} vs {name2}:")
    print("  Estatística:", mannwhitney_result.statistic)
    print("  p-valor:", mannwhitney_result.pvalue)
    if mannwhitney_result.pvalue < alpha:
        print('  Isso indica que os valores das acuracias apuradas são significativamente diferentes.')
    else:
        print('  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.')



Comparação MLP vs DT:
  Estatística: 17.0
  p-valor: 0.01237160821796184
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação MLP vs SVM:
  Estatística: 17.5
  p-valor: 0.014079121613493622
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação MLP vs KNN:
  Estatística: 20.0
  p-valor: 0.0235556726630649
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação DT vs SVM:
  Estatística: 37.5
  p-valor: 0.3614420145078252
  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.
Comparação DT vs KNN:
  Estatística: 68.0
  p-valor: 0.18305346808287726
  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.
Comparação SVM vs KNN:
  Estatística: 90.5
  p-valor: 0.0024133553534221547
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
